# CREAETING THE AUGMENTED DATASET

In [4]:
!rm -r venv_txtatck
!python3.7 -m venv venv_txtatck
!pip install textattack -q

In [5]:
from textattack.augmentation.recipes import EasyDataAugmenter
easyAugmenter = EasyDataAugmenter(pct_words_to_swap=0.5, transformations_per_example=1)

[nltk_data] Downloading package omw-1.4 to /home/maxwell/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
import pandas as pd
import numpy as np

In [9]:
data = pd.read_csv(
    'Sentiment_Dataset/SemEval17/Clean/semeval.csv', 
    encoding = 'utf-8'
)

In [10]:
data['augmented'] = np.nan

In [11]:
data

,polarity,text,augmented
0,NEU,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...",NaN
1,NEU,Order Go Set a Watchman in store or through ou...,NaN
2,NEG,If these runway renovations at the airport pre...,NaN
3,NEU,If you could ask an onstage interview question...,NaN
4,POS,A portion of book sales from our Harper Lee/Go...,NaN
...,...,...,...
20627,NEU,@ShaquilleHoNeal from what I think you're aski...,NaN
20628,POS,"Iran ranks 1st in liver surgeries, Allah bless...",NaN
20629,NEU,Hours before he arrived in Saudi Arabia on Tue...,NaN
20630,NEG,@VanityFair Alex Kim Kardashian worth how to ...,NaN


In [13]:
from tqdm import tqdm
for i in tqdm(range(len(data))):
    data["augmented"][i] = easyAugmenter.augment(data["text"][i])[0]

  0%|          | 0/20632 [00:00<?, ?it/s]/tmp/ipykernel_36943/4289361308.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["augmented"][i] = easyAugmenter.augment(data["text"][i])[0]
100%|██████████| 20632/20632 [34:10<00:00, 10.06it/s]


In [16]:
data

,polarity,text,augmented
0,NEU,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...","Picturehouse's,, 'Roger: The - opening 29 Sept..."
1,NEU,Order Go Set a Watchman in store or through ou...,tell proceed countersink a watchman in stock o...
2,NEG,If these runway renovations at the airport pre...,If these rails restoration at the aerodrome fo...
3,NEU,If you could ask an onstage interview question...,"If ask an onstage Miss USA tomorrow, it?"
4,POS,A portion of book sales from our Harper Lee/Go...,ampere dowry of Koran sales from our Harper le...
...,...,...,...
20627,NEU,@ShaquilleHoNeal from what I think you're aski...,@asking what from in Future no ShaquilleHoNeal...
20628,POS,"Iran ranks 1st in liver surgeries, Allah bless...",Iran outrank ranks 1st first in liver surgerie...
20629,NEU,Hours before he arrived in Saudi Arabia on Tue...,Hours Tuesday he Arabia mercilessly Syria's in...
20630,NEG,@VanityFair Alex Kim Kardashian worth how to ...,@VanityFair Alex Kim Kardashian Worth how to ...


In [18]:
# !mkdir Sentiment_Dataset/SemEval17/Augmented
data.to_csv("./Sentiment_Dataset/SemEval17/Augmented/augmented_SemEval17.csv",encoding='utf-8')

# CREATING THE PREDICTIONS

In [20]:
# !python3 -m venv venv_pysentimiento
!pip install pysentimiento
!pip install -U scikit-learn

  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 9.2 MB/s eta 0:00:000:00:01m eta 0:00:01m
  Using cached emoji-1.7.0.tar.gz (175 kB)
  Preparing metadata (setup.py) ... done
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached pydantic-1.10.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached srsly-2.4.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (492 kB)
  Using cached pathy-0.10.1-py3-none-any.whl (48 kB)
  Using cached murmurhash-1.0.9-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21 kB)
  Using cached typer-0.7.0-py3-none-any.whl (38 kB)
  Using cached preshed-3.0.8-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (124 kB)
  Using cached wasabi-1.1.1-py3-none-any.whl (27 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Usi

In [1]:
from pysentimiento import create_analyzer
sentiment_analyzer = create_analyzer(task="sentiment", lang="en")

/home/maxwell/Thesis/restructure/venv_pysentimiento/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
data = pd.read_csv("Sentiment_Dataset/SemEval17/Augmented/augmented_SemEval17.csv", encoding='utf-8')

In [4]:
data['predicted_polarity'] = np.nan
data['augmented_polarity'] = np.nan

In [5]:
data

,Unnamed: 0,polarity,text,augmented,predicted_polarity,augmented_polarity
0,0,NEU,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...","Picturehouse's,, 'Roger: The - opening 29 Sept...",NaN,NaN
1,1,NEU,Order Go Set a Watchman in store or through ou...,tell proceed countersink a watchman in stock o...,NaN,NaN
2,2,NEG,If these runway renovations at the airport pre...,If these rails restoration at the aerodrome fo...,NaN,NaN
3,3,NEU,If you could ask an onstage interview question...,"If ask an onstage Miss USA tomorrow, it?",NaN,NaN
4,4,POS,A portion of book sales from our Harper Lee/Go...,ampere dowry of Koran sales from our Harper le...,NaN,NaN
...,...,...,...,...,...,...
20627,20627,NEU,@ShaquilleHoNeal from what I think you're aski...,@asking what from in Future no ShaquilleHoNeal...,NaN,NaN
20628,20628,POS,"Iran ranks 1st in liver surgeries, Allah bless...",Iran outrank ranks 1st first in liver surgerie...,NaN,NaN
20629,20629,NEU,Hours before he arrived in Saudi Arabia on Tue...,Hours Tuesday he Arabia mercilessly Syria's in...,NaN,NaN
20630,20630,NEG,@VanityFair Alex Kim Kardashian worth how to ...,@VanityFair Alex Kim Kardashian Worth how to ...,NaN,NaN


In [6]:
for i in tqdm(range(len(data))):

    data['predicted_polarity'][i] = sentiment_analyzer.predict(data['text'][i]).output
    data['augmented_polarity'][i] = sentiment_analyzer.predict(data['augmented'][i]).output

  0%|          | 0/20632 [00:00<?, ?it/s]/home/maxwell/Thesis/restructure/venv_pysentimiento/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/maxwell/Thesis/restructure/venv_pysentimiento/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/maxwell/Thesis/restructure/venv_pysentimiento/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying t

In [7]:
data.to_csv('./Sentiment_Dataset/SemEval17/Augmented/predicted_semeval17.csv', encoding='utf-8',index=False)

# EVALUATION

In [8]:
from sklearn.metrics import classification_report, confusion_matrix

In [9]:
data

,Unnamed: 0,polarity,text,augmented,predicted_polarity,augmented_polarity
0,0,NEU,"Picturehouse's, Pink Floyd's, 'Roger Waters: T...","Picturehouse's,, 'Roger: The - opening 29 Sept...",NEU,NEU
1,1,NEU,Order Go Set a Watchman in store or through ou...,tell proceed countersink a watchman in stock o...,NEU,NEU
2,2,NEG,If these runway renovations at the airport pre...,If these rails restoration at the aerodrome fo...,NEG,NEG
3,3,NEU,If you could ask an onstage interview question...,"If ask an onstage Miss USA tomorrow, it?",NEU,NEU
4,4,POS,A portion of book sales from our Harper Lee/Go...,ampere dowry of Koran sales from our Harper le...,POS,NEU
...,...,...,...,...,...,...
20627,20627,NEU,@ShaquilleHoNeal from what I think you're aski...,@asking what from in Future no ShaquilleHoNeal...,NEU,NEU
20628,20628,POS,"Iran ranks 1st in liver surgeries, Allah bless...",Iran outrank ranks 1st first in liver surgerie...,POS,POS
20629,20629,NEU,Hours before he arrived in Saudi Arabia on Tue...,Hours Tuesday he Arabia mercilessly Syria's in...,NEG,NEG
20630,20630,NEG,@VanityFair Alex Kim Kardashian worth how to ...,@VanityFair Alex Kim Kardashian Worth how to ...,NEG,NEU


In [10]:
print("REPORT")
print(classification_report(data['polarity'], data["predicted_polarity"], target_names=['NEG','NEU','POS']))

print("CONFUSSION MATRIX")
print(confusion_matrix(data['polarity'], data["predicted_polarity"]))


REPORT
              precision    recall  f1-score   support

         NEG       0.79      0.88      0.83      3231
         NEU       0.88      0.82      0.85     10342
         POS       0.84      0.88      0.86      7059

    accuracy                           0.85     20632
   macro avg       0.84      0.86      0.85     20632
weighted avg       0.85      0.85      0.85     20632

CONFUSSION MATRIX
[[2839  373   19]
 [ 699 8480 1163]
 [  34  805 6220]]


In [11]:
print("REPORT")
print(classification_report(data['polarity'], data["augmented_polarity"], target_names=['NEG','NEU','POS']))

print("CONFUSSION MATRIX")
print(confusion_matrix(data['polarity'], data["augmented_polarity"]))

REPORT
              precision    recall  f1-score   support

         NEG       0.66      0.61      0.63      3231
         NEU       0.69      0.85      0.76     10342
         POS       0.83      0.57      0.68      7059

    accuracy                           0.72     20632
   macro avg       0.73      0.68      0.69     20632
weighted avg       0.73      0.72      0.71     20632

CONFUSSION MATRIX
[[1961 1209   61]
 [ 789 8801  752]
 [ 200 2834 4025]]
